In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.linear_model as sk
import math
from sklearn import preprocessing
from scipy import stats

    Dropping features that seem irrelevant (such as ID) or that gives some kind of repeated information (such as usd_pledfed_real, usd_goal_real and category).

In [2]:
# df = pd.read_csv('ks-projects-201612.csv')
df2018 = pd.read_csv('ks-projects-201801.csv')

In [3]:
df2018.drop(['ID', 'name', 'pledged', 'usd_pledged_real', 'usd_goal_real', 'category'], axis=1, inplace=True)
print df2018.shape

(378661, 9)


In [4]:
label_column = 'state'
time_column = 'time_elapsed'

df2018['launched'] = pd.to_datetime(df2018['launched'])
df2018['deadline'] = pd.to_datetime(df2018['deadline'])
df2018[time_column] = (df2018.deadline - df2018.launched).astype('timedelta64[h]')
# print df2018['launched'].head(10)
# print(df2018[time_column].head(15))

df2018.drop(['launched', 'deadline'], axis=1, inplace=True)

In [5]:
columns = ['main_category','currency', 'country']

In [6]:
def getDummies(df):
    return pd.get_dummies(df)

In [7]:
dummies = getDummies(df2018[columns])
dummies.head()

,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,main_category_Fashion,main_category_Film & Video,main_category_Food,main_category_Games,main_category_Journalism,...,country_JP,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
df2018.drop(columns, axis=1, inplace=True)

In [9]:
df2018 = pd.concat([df2018,dummies], axis=1)
df2018.shape

(378661, 57)

In [10]:
#df2018[columns] = df2018[columns].apply(lambda x: pd.factorize(x)[0])
# df2018 = df2018.apply(lambda x: pd.factorize(x)[0])

#print df2018.head()

In [11]:
df2018_copy = df2018
df2018_copy.head()

,goal,state,backers,usd pledged,time_elapsed,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,...,country_JP,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,1000.0,failed,0,0.0,1403.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30000.0,failed,15,100.0,1435.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,45000.0,failed,3,220.0,1079.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5000.0,failed,1,1.0,716.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,19500.0,canceled,14,1283.0,1335.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
df2018.dropna(inplace=True)

In [13]:
# df2018[label_column] = df2018[[label_column]].apply(lambda x: pd.factorize(x)[0])
y_target = df2018[label_column]
df2018 = df2018.drop([label_column], axis=1)

In [14]:
df2018.head()

,goal,backers,usd pledged,time_elapsed,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,main_category_Fashion,...,country_JP,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,1000.0,0,0.0,1403.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30000.0,15,100.0,1435.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,45000.0,3,220.0,1079.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5000.0,1,1.0,716.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,19500.0,14,1283.0,1335.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
def normalizeDF(df):
    x = df.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled, columns=df.columns)
    return df

In [17]:
df2018_norm = normalizeDF(df2018)
df2018_norm.head()

,goal,backers,usd pledged,time_elapsed,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,main_category_Fashion,...,country_JP,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,0.000010,0.000000,0.000000e+00,0.003492,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000300,0.000068,4.916666e-06,0.003572,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.000450,0.000014,1.081666e-05,0.002686,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.000050,0.000005,4.916666e-08,0.001782,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.000195,0.000064,6.308082e-05,0.003323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
def showCorr(df):
    corr = df.corr()
    plt.figure(figsize=(10, 6))
    sns.heatmap(corr, cbar=None, annot=True, cmap='Blues')
    plt.show()

In [19]:
# Too many features
#showCorr(df2018)

In [20]:
def showPairPlot(df):
    sns.pairplot(df)
    plt.show()

In [21]:
#Too many features
# showPairPlot(df2018)

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#In general a good idea is to scale the data
scaler = StandardScaler()
scaler.fit(df2018)
X=scaler.transform(df2018)    

pca = PCA()
x_new = pca.fit_transform(X)

In [23]:
x_new

array([[ -3.70135889e+00,  -3.66355713e+00,   1.26068917e-01, ...,
          2.66545094e-15,  -6.91373253e-18,   5.16178925e-17],
       [  1.10314752e+00,  -1.84447487e-01,   4.78597929e-02, ...,
         -1.05275771e-15,  -1.78117215e-17,   2.78166553e-16],
       [  1.10170229e+00,  -1.86930981e-01,   4.84380023e-02, ...,
          5.91103616e-15,  -7.66935549e-19,  -1.71071154e-15],
       ..., 
       [  1.10182757e+00,  -1.87316237e-01,   4.86172931e-02, ...,
         -7.94797232e-17,   1.47970625e-19,   3.52641261e-17],
       [  7.75243347e-01,   2.92167552e-01,  -1.87328940e-01, ...,
          9.39463150e-18,   2.07556433e-20,   7.13742133e-17],
       [  1.01974145e+00,  -7.30739401e-02,   2.91026888e-02, ...,
         -6.52542684e-17,  -3.28010589e-18,   4.27928420e-17]])

In [29]:
pca_variance = pca.explained_variance_ratio_
pca_variance

array([  6.70750817e-02,   3.92611647e-02,   3.77684203e-02,
         3.72268891e-02,   3.66795427e-02,   3.65945798e-02,
         3.65546849e-02,   3.65145270e-02,   3.64729915e-02,
         3.64592183e-02,   3.64432057e-02,   3.64348694e-02,
         3.63770683e-02,   3.12798924e-02,   2.15190756e-02,
         2.05661454e-02,   2.00730121e-02,   1.99544155e-02,
         1.98004618e-02,   1.95477254e-02,   1.94076631e-02,
         1.91579450e-02,   1.88348456e-02,   1.87353010e-02,
         1.87077431e-02,   1.86150672e-02,   1.84573545e-02,
         1.83720126e-02,   1.83279356e-02,   1.82913672e-02,
         1.82425049e-02,   1.82346439e-02,   1.82022124e-02,
         1.81781564e-02,   1.81156001e-02,   1.80778128e-02,
         1.80456341e-02,   1.79148094e-02,   5.47841931e-03,
         7.45463970e-32,   6.56456232e-32,   4.97520650e-32,
         3.86358549e-32,   3.69908724e-32,   3.33789687e-32,
         2.94304392e-32,   2.60997923e-32,   1.96839136e-32,
         1.86783693e-32,

In [33]:
sum(pca_variance[0:39])

0.99999999999999978

In [37]:
pca = PCA(n_components=39)
x_new = pca.fit_transform(X)

In [38]:
x_new

array([[ -3.70135889e+00,  -3.66355713e+00,   1.26068917e-01, ...,
         -5.34441973e-02,  -1.44126929e-01,   1.40443890e-02],
       [  1.10314752e+00,  -1.84447487e-01,   4.78597929e-02, ...,
         -1.58656576e-01,   1.25319568e-01,  -1.40771034e-02],
       [  1.10170229e+00,  -1.86930981e-01,   4.84380023e-02, ...,
         -1.94638874e-01,   5.54150644e-02,  -3.45565907e-03],
       ..., 
       [  1.10182757e+00,  -1.87316237e-01,   4.86172931e-02, ...,
         -2.10104532e-01,   7.26366294e-02,  -3.66768213e-03],
       [  7.75243347e-01,   2.92167552e-01,  -1.87328940e-01, ...,
          1.45417038e-02,   1.00458840e+00,  -2.53048756e-02],
       [  1.01974145e+00,  -7.30739401e-02,   2.91026888e-02, ...,
         -9.16042956e-02,  -7.19624889e-02,  -1.78520705e-03]])

In [42]:
nDF2018 = pd.DataFrame(np.round(x_new, 4))
nDF2018.head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,-3.7014,-3.6636,0.1261,-0.2125,-0.3506,0.0542,0.0249,-0.0233,-0.0260,0.0275,...,-0.0504,0.0292,0.0628,0.1202,-0.0440,0.0596,0.4055,-0.0534,-0.1441,0.0140
1,1.1031,-0.1844,0.0479,0.0264,0.0090,-0.0609,-0.0196,0.0087,0.0009,0.0076,...,-0.0207,-0.0573,-0.0215,0.0411,0.0149,0.0348,0.2337,-0.1587,0.1253,-0.0141
2,1.1017,-0.1869,0.0484,0.0268,0.0083,-0.0620,-0.0181,0.0099,0.0019,0.0040,...,-0.0191,-0.0574,-0.0371,0.0121,-0.0024,-0.0283,0.0537,-0.1946,0.0554,-0.0035
3,1.1902,-0.0477,0.0206,-0.0268,0.0269,-0.0466,0.0037,-0.0138,0.0216,0.0137,...,-0.0110,-0.0256,0.0068,-0.0631,-0.0565,-0.0915,-0.1370,-0.0589,-0.2930,0.0093
4,1.1030,-0.1852,0.0479,0.0263,0.0081,-0.0607,-0.0191,0.0091,0.0011,0.0065,...,-0.0211,-0.0583,-0.0281,0.0326,0.0096,0.0176,0.1833,-0.1772,0.1129,-0.0026


    First, let's try using a linear model.

In [43]:
def clf_score(df, y_target, clf):
    X = np.array(df)
    y = y_target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=89, shuffle=True)

    print "Starting..."

    clf.fit(X_train, y_train)

    print "Finished!"

    return clf.score(X_test, y_test)

In [44]:
score = clf_score(df2018, y_target, sk.LogisticRegression())
score

Starting...
Finished!


0.7757459352033399

In [45]:
score_norm = clf_score(df2018_norm, y_target, sk.LogisticRegression())
score_norm

Starting...
Finished!


0.56306937164045723

In [46]:
score_pca = clf_score(nDF2018, y_target, sk.LogisticRegression())
score_pca

Starting...
Finished!


0.72998279380577002

In [52]:
from sklearn.neural_network import MLPClassifier

mlp_score = clf_score(df2018, y_target, MLPClassifier(alpha=1))
mlp_score

Starting...
Finished!


0.83109919571045576

In [53]:
mlp_score_norm = clf_score(df2018_norm, y_target, MLPClassifier(alpha=1))
mlp_score_norm

Starting...
Finished!


0.5508383017886439

In [54]:
mlp_score_pca = clf_score(nDF2018, y_target, MLPClassifier(alpha=1))
mlp_score_pca

Starting...
Finished!


0.73578488255772079

In [ ]:
from sklearn.svm import SVC

svm_score = clf_score(df2018_norm, y_target, SVC())
svm_score_norm = clf_score(df2018_norm, y_target, SVC())
svm_score_pca = clf_score(df2018_norm, y_target, SVC())

Starting...


In [ ]:
print svm_score
print svm_score_norm
print svm_score_pca